In [1]:
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.restaurants

In [2]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

def convertRestaurant(restaurant):
    return {
        "name": restaurant["name"],
        "cuisine":restaurant["cuisine"],
        "position": toGeoJSON(restaurant["address"]["coord"])
    }

cursor = db.restaurants.find({"address.coord":{"$size":2}},{"_id":0})
geoRestaurants = list(map(lambda r: convertRestaurant(r),cursor))


In [3]:
import pandas as pd

df = pd.DataFrame(geoRestaurants)
df.head()

Empty DataFrame
Columns: []
Index: []

In [4]:
df.to_json("rest_with_coords.json", orient="records", lines=True)

In [5]:
cursor = db.clean.find({
  "position": {
    "$near": {
      "$geometry": {
        "type": "Point",
        "coordinates": [-74.1080159, 40.7046139]
      },
      "$minDistance": 0,
      "$maxDistance":10000
    }
  }
})

import pandas as pd
df = pd.DataFrame(list(cursor))
df.head()

Empty DataFrame
Columns: []
Index: []

In [6]:
coords_df = pd.DataFrame(pd.DataFrame(df["position"].values.flatten().tolist())["coordinates"].values.tolist())
clean_df = pd.concat([df,coords_df],axis=1)
clean_df.drop(['_id',"position"], inplace=True, axis=1)
clean_df = clean_df.rename({0:"long",1:"lat"}, axis=1)


KeyError: 'position'

In [ ]:
clean_df.to_json('visualize_restaurants.json', orient="records")